# Day 09. Exercise 02
# Metrics

## 0. Imports

In [1]:
# This is an display config snippet!
import pandas as pd
from IPython.core.display import display, HTML
pd.options.display.float_format = '{:,.4f}'.format

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.expand_frame_repr', False)
display(HTML("<style>.container { width:95% !important; }</style>"))

/Users/a19028558/Desktop/transfer_accounts/akovyaz/learning_myself/env_school/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score, recall_score
import numpy as np
from sklearn.utils._testing import ignore_warnings
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import pickle
from sklearn.model_selection import GridSearchCV
from tqdm.auto import tqdm
from itertools import product
from collections import defaultdict

## 1. Preprocessing

1. Create the same dataframe as in the previous exercise.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [3]:
df = pd.read_csv('../data/dayofweek-not-scaled.csv')

In [4]:
scaler = StandardScaler()

In [5]:
X = np.hstack([scaler.fit_transform(df[['numTrials', 'hour']]), df.drop(columns=['numTrials', 'hour', 'dayofweek'])])
y = df['dayofweek'].values

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=21, test_size=0.2)

## 2. SVM

1. Use the best parameters from the previous exercise and train the model of SVM.
2. You need to calculate `accuracy`, `precision`, `recall`, `ROC AUC`.

 - `precision` and `recall` should be calculated for each class (use `average='weighted'`)
 - `ROC AUC` should be calculated for each class against any other class (all possible pairwise combinations) and then weighted average should be applied for the final metric
 - the code in the cell should display the result as below:

```
accuracy is 0.88757
precision is 0.89267
recall is 0.88757
roc_auc is 0.97878
```

In [7]:
svc_params = {'C': 10, 'class_weight': None, 'gamma': 'scale', 'kernel': 'rbf'}

In [8]:
svc = SVC(random_state=21, probability=True, **svc_params)

In [9]:
%%time
svc.fit(X_train, y_train)

CPU times: user 722 ms, sys: 5.78 ms, total: 728 ms
Wall time: 1.03 s


SVC(C=10, probability=True, random_state=21)

In [10]:
y_pred = svc.predict(X_test)

In [11]:
print(f"accuracy is {accuracy_score(y_test, y_pred)}")
print(f"precision is {precision_score(y_test, y_pred, average='weighted')}")
print(f"recall is {recall_score(y_test, y_pred, average='weighted')}")
print(f"roc_auc is {roc_auc_score(y_test, svc.predict_proba(X_test), average='weighted', multi_class='ovo')}")

accuracy is 0.8964497041420119
precision is 0.8987502097411376
recall is 0.8964497041420119
roc_auc is 0.9826366175798507


## 3. Decision tree

1. The same task for decision tree

In [12]:
tree_params = {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 25}

In [13]:
tree = DecisionTreeClassifier(random_state=21, **tree_params)

In [14]:
%%time
tree.fit(X_train, y_train)

CPU times: user 7.91 ms, sys: 1.52 ms, total: 9.44 ms
Wall time: 8.01 ms


DecisionTreeClassifier(class_weight='balanced', max_depth=25, random_state=21)

In [15]:
y_pred = tree.predict(X_test)

In [16]:
print(f"accuracy is {accuracy_score(y_test, y_pred)}")
print(f"precision is {precision_score(y_test, y_pred, average='weighted')}")
print(f"recall is {recall_score(y_test, y_pred, average='weighted')}")
print(f"roc_auc is {roc_auc_score(y_test, svc.predict_proba(X_test), average='weighted', multi_class='ovo')}")

accuracy is 0.8994082840236687
precision is 0.9010905154871189
recall is 0.8994082840236687
roc_auc is 0.9826366175798507


## 4. Random forest

1. The same task for random forest.

In [17]:
forest_params = {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 30, 'n_estimators': 50}

In [18]:
forest = RandomForestClassifier(random_state=21, **forest_params)

In [19]:
%%time
forest.fit(X_train, y_train)

CPU times: user 182 ms, sys: 4.75 ms, total: 187 ms
Wall time: 239 ms


RandomForestClassifier(class_weight='balanced', max_depth=30, n_estimators=50,
                       random_state=21)

In [20]:
y_pred = forest.predict(X_test)

In [21]:
print(f"accuracy is {accuracy_score(y_test, y_pred)}")
print(f"precision is {precision_score(y_test, y_pred, average='weighted')}")
print(f"recall is {recall_score(y_test, y_pred, average='weighted')}")
print(f"roc_auc is {roc_auc_score(y_test, svc.predict_proba(X_test), average='weighted', multi_class='ovo')}")

accuracy is 0.9349112426035503
precision is 0.9360699474016719
recall is 0.9349112426035503
roc_auc is 0.9826366175798507


## 5. Predictions

1. Choose the best model.
2. Analyze: for which `weekday` your model makes the most errors (in % of the total number of samples of that class in your full dataset), for which `labname` and for which `users`.
3. Save the model.

Лучшая модель - random forest

In [22]:
pd.value_counts(y_test[y_test != y_pred])/pd.value_counts(y_test)

0   0.2593
1   0.0545
2   0.0667
3   0.0375
4   0.1429
5   0.0741
6      NaN
dtype: float64

In [23]:
with open('../data/ex02_random_forest.pkl', 'wb') as f:
    pickle.dump(forest, f)

## 6. Function

1. Write a function that takes a list of different models and a corresponding list of parameters (dicts) and returns a dict that contains all the 4 metrics for each model.

In [24]:
def magic(estimators, params):
    result_metrics = defaultdict(dict)
    assert len(estimators) == len(params)
    for estimator, model_params in zip(estimators, params):
        estimator = estimator.set_params(**model_params).fit(X_train, y_train)
        y_pred = estimator.predict(X_test)
        estimator_name = type(estimator).__name__
        result_metrics['accuracy'][estimator_name] = accuracy_score(y_test, y_pred)
        result_metrics['precision'][estimator_name] = precision_score(y_test, y_pred, average='weighted')
        result_metrics['recall'][estimator_name] = recall_score(y_test, y_pred, average='weighted')
        result_metrics['roc_auc'][estimator_name] = roc_auc_score(y_test, estimator.predict_proba(X_test), average='weighted', multi_class='ovo')
    return dict(result_metrics)

In [25]:
magic([SVC(random_state=21, probability=True), DecisionTreeClassifier(random_state=21), RandomForestClassifier(random_state=21)], [svc_params, tree_params, forest_params])

{'accuracy': {'SVC': 0.8964497041420119,
  'DecisionTreeClassifier': 0.8994082840236687,
  'RandomForestClassifier': 0.9349112426035503},
 'precision': {'SVC': 0.8987502097411376,
  'DecisionTreeClassifier': 0.9010905154871189,
  'RandomForestClassifier': 0.9360699474016719},
 'recall': {'SVC': 0.8964497041420119,
  'DecisionTreeClassifier': 0.8994082840236687,
  'RandomForestClassifier': 0.9349112426035503},
 'roc_auc': {'SVC': 0.9826366175798507,
  'DecisionTreeClassifier': 0.9387933106378391,
  'RandomForestClassifier': 0.9857175724281403}}